In [1]:
import pandas as pd
import numpy as np

In [4]:
with open('input.txt', 'r') as f:
    lines = f.readlines()
data = [line[:-1] for line in lines]
data

['313213123212200312011243203120214010202554420335045116203101005212525131015305511140012431022113113',
 '222021333112322244231542054023354511304431216312534453560105660253445311020244233130144313440030322',
 '200012220300132310233100025524032003355144536126232161330106115213441450552235514000243120210031130',
 '102111124044033113241415155401056014506203413216651520206613342353052505445000321043121021430440021',
 '221134021010031320101542213340113552161113350123516154134654046250223033000413443321231233142213010',
 '112241233114245202523201433013460320664034202602144006022621253536213222400131120345151014014131133',
 '132211130114141401325141212460540656525106651152012063404633651340223361443201520054113541144043031',
 '323140320024314005210155316202152552621415660063112177564603241212312025036551350313225504314404112',
 '014013242041230510442265205625331115540222744516266167254216354043533020346502032534231121044441220',
 '23341403010131525533512605526314451211177746461156521615744167

In [10]:
X = np.zeros((len(data), len(data)), dtype=int)

for i, row in enumerate(data):
    ary = np.array([int(x) for x in list(row)], dtype=int)
    X[i, :] = ary
    
X.shape # (row, col)

(99, 99)

In [64]:
def get_vis(X):
    left_vis = np.zeros_like(X, dtype=int)
    left_vis[:, 0] = 1
    
    up_vis = np.zeros_like(X, dtype=int)
    up_vis[0, :] = 1
    
    right_vis = np.zeros_like(X, dtype=int)
    right_vis[:, -1] = 1
    
    down_vis = np.zeros_like(X, dtype=int)
    down_vis[-1, :] = 1
    
    # left
    for ri in np.arange(X.shape[0]):
        for ci in np.arange(X.shape[1]):
            if ci == 0:
                continue            
            diff = np.max(X[ri, :ci]) < X[ri, ci]
            left_vis[ri, ci] = int(diff)
    
    # up
    for ci in np.arange(X.shape[1]):
        for ri in np.arange(X.shape[0]):
            if ri == 0:
                continue
            diff = np.max(X[:ri, ci]) < X[ri, ci]
            up_vis[ri, ci] = int(diff)
            
    # right
    for ri in np.arange(X.shape[0]):
        for ci in np.arange(X.shape[1]):
            if ci == 0:
                continue            
            diff = np.max(X[ri, -1-ci+1:]) < X[ri, -1-ci]
            right_vis[ri, -1-ci] = int(diff)
    
    # down
    for ci in np.arange(X.shape[1]):
        for ri in np.arange(X.shape[0]):
            if ri == 0:
                continue
            diff = np.max(X[-1-ri+1:, ci]) < X[-1-ri, ci]
            down_vis[-1-ri, ci] = int(diff)
    
    return left_vis, up_vis, right_vis, down_vis
        
left_vis, up_vis, right_vis, down_vis = get_vis(X)

print('--- from left')

print(left_vis[0, :30])
print(X[0, :30])

print('--- from up')

print(up_vis[:30, 0])
print(X[:30, 0])

print('--- from right')

print(right_vis[0, -30:])
print(X[0, -30:])

print('--- from down')

print(down_vis[-30:, 0])
print(X[-30:, 0])

--- from left
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[3 1 3 2 1 3 1 2 3 2 1 2 2 0 0 3 1 2 0 1 1 2 4 3 2 0 3 1 2 0]
--- from up
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[3 2 2 1 2 1 1 3 0 2 1 0 3 0 0 4 4 3 2 2 3 0 1 0 1 4 1 3 3 0]
--- from right
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1]
[1 3 1 0 1 5 3 0 5 5 1 1 1 4 0 0 1 2 4 3 1 0 2 2 1 1 3 1 1 3]
--- from down
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1]
[4 0 3 4 5 3 2 2 0 4 1 2 0 0 0 2 3 4 2 3 1 1 3 0 2 2 1 0 0 1]


In [66]:
np.count_nonzero(right_vis + left_vis + up_vis + down_vis)

1715

In [93]:
score_map = np.zeros_like(X, dtype=int)

for ri in np.arange(X.shape[0]):
    for ci in np.arange(X.shape[1]):
        current_cell = X[ri, ci]
            
        def get_score(part, ri, ci):
            if part.shape[0] == 0:
                return 0
            
            part_score = np.where(part >= X[ri, ci])[0]
            if part_score.shape[0] == 0:
                return len(part)
            else:
                return part_score[0] + 1

        left_score = get_score(np.flip(X[ri, :ci], 0), ri, ci)
        right_score = get_score(np.flip(X[:ri, ci], 0), ri, ci)
        top_score = get_score(X[ri, ci+1:], ri, ci)
        down_score = get_score(X[ri+1:, ci], ri, ci)   
        
        score_map[ri, ci] = left_score * right_score * top_score * down_score
            
np.max(score_map)

374400